# **IRIS CIANNA**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Deyht/AI_astro_ED_AAIF/blob/main/codes/mlp/CIANNA/iris_colab_notebook.ipynb)

---


**Link to the CIANNA github repository**
https://github.com/Deyht/CIANNA

### **CIANNA installation**

#### Query GPU allocation and properties

If nvidia-smi fail, it might indicate that you launched the colab session whithout GPU reservation.  
To change the type of reservation go to "Runtime"->"Change runtime type" and select "GPU" as your hardware accelerator.

In [1]:
%%shell

nvidia-smi

cd /content/

git clone https://github.com/NVIDIA/cuda-samples/

cd /content/cuda-samples/Samples/1_Utilities/deviceQuery/

cmake CMakeLists.txt

make SMS="50 60 70 80"

./deviceQuery | grep Capability | cut -c50- > ~/cuda_infos.txt
./deviceQuery | grep "CUDA Driver Version / Runtime Version" | cut -c57- >> ~/cuda_infos.txt

cd ~/

Fri Dec 13 09:28:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

If you are granted a GPU that supports high FP16 compute scaling (e.g the Tesla T4), it is advised to change the mixed_precision parameter in the prediction to "FP16C_FP32A".  
See the detail description on mixed precision support with CIANNA on the [Systeme Requirements](https://github.com/Deyht/CIANNA/wiki/1\)-System-Requirements) wiki page.

#### Clone CIANNA git repository

In [2]:
%%shell

cd /content/

git clone https://github.com/Deyht/CIANNA

cd CIANNA

Cloning into 'CIANNA'...
remote: Enumerating objects: 2120, done.
remote: Counting objects: 100% (549/549), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 2120 (delta 389), reused 518 (delta 364), pack-reused 1571 (from 1)
Receiving objects: 100% (2120/2120), 62.13 MiB | 15.24 MiB/s, done.
Resolving deltas: 100% (1571/1571), done.


#### Compiling CIANNA for the allocated GPU generation

There is no guaranteed forward or backward compatibility between Nvidia GPU generation, and some capabilities are generation specific. For these reasons, CIANNA must be provided the platform GPU generation at compile time.
The following cell will automatically update all the necessary files based on the detected GPU, and compile CIANNA.

In [3]:
%%shell

cd /content/CIANNA

mult="10"
cat ~/cuda_infos.txt
comp_cap="$(sed '1!d' ~/cuda_infos.txt)"
cuda_vers="$(sed '2!d' ~/cuda_infos.txt)"

lim="11.1"
old_arg=$(awk '{if ($1 < $2) print "-D CUDA_OLD";}' <<<"${cuda_vers} ${lim}")

sm_val=$(awk '{print $1*$2}' <<<"${mult} ${comp_cap}")

gen_val=$(awk '{if ($1 >= 80) print "-D GEN_AMPERE"; else if($1 >= 70) print "-D GEN_VOLTA";}' <<<"${sm_val}")

sed -i "s/.*arch=sm.*/\\t\tcuda_arg=\"\$cuda_arg -D CUDA -D comp_CUDA -lcublas -lcudart -arch=sm_$sm_val $old_arg $gen_val\"/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" src/python_module_setup.py

./compile.cp CUDA PY_INTERF

mv src/build/lib.linux-x86_64-* src/build/lib.linux-x86_64

7.5
12.2
USE_CUDA
BUILD PY_INTERF
#####  End of CUDA compilation  #####
#####  End of main program compilation  #####
#####  End of link edition and executable creation  #####
USE_CUDA
running build
running build_ext
building 'CIANNA' extension
creating build/temp.linux-x86_64-cpython-310
x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -DMAX_LAYERS_NB=200 -DMAX_NETWORKS_NB=10 -DCUDA=1 -DCUDA_THREADS_PER_BLOCKS=256 -DNone -I/usr/local/cuda-12.2/include -I/usr/local/lib/python3.10/dist-packages/numpy/core/include -I/usr/include/python3.10 -c python_module.c -o build/temp.linux-x86_64-cpython-310/python_module.o
creating build/lib.linux-x86_64-cpython-310
x86_64-linux-gnu-gcc -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 build/temp.linux-x86_64-cpython-310/python_module.o conv_layer.o dense_layer.o pool_layer.o norm_layer.o lrn_layer.o a


**IMPORTANT NOTE**   
CIANNA is mainly used in a script fashion and was not designed to run in notebooks. Every cell code that directly invokes CIANNA functions must be run as a script to avoid possible errors.  
To do so, the cell must have the following structure.

```
%%shell

cd /content/CIANNA

python3 - <<EOF

[... your python code ...]

EOF
```

This syntax allows one to easily edit python code in the notebook while running the cell as a script. Note that all the notebook variables can not be accessed by the cell in this context.


In [5]:
%%shell

cd /content/

#Manually upload the directory to github if not yet opened
git clone https://github.com/Deyht/AI_astro_ED_AAIF

Cloning into 'AI_astro_ED_AAIF'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 126 (delta 36), reused 56 (delta 26), pack-reused 49 (from 1)
Receiving objects: 100% (126/126), 9.69 MiB | 4.80 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [8]:

%%shell

cd /content/

python3 - <<EOF

import numpy as np
import sys, glob

sys.path.insert(0,glob.glob('/content/CIANNA/src/build/lib.*/')[-1])
import CIANNA as cnn


def i_ar(int_list):
	return np.array(int_list, dtype="int")

def f_ar(float_list):
	return np.array(float_list, dtype="float32")

nb_train = 100 # over 150


######################### ##########################
#          Loading data and pre process
######################### ##########################

raw_data = np.loadtxt("/content/AI_astro_ED_AAIF/codes/data/iris.data")


nb_dat = np.shape(raw_data)[0]
in_dim = np.shape(raw_data)[1] - 1
out_dim = 3

input = np.append(raw_data[:,:in_dim], -1.0*np.ones((nb_dat,1)), axis=1)


targ = np.zeros((nb_dat,out_dim))
for i in range(0,nb_dat):
	targ[i,int(raw_data[i,in_dim])] = 1.0


input[:,:-1] -= np.mean(input[:,:-1], axis = 0)
input[:,:-1] /= np.max(np.abs(input[:,:-1]), axis = 0)


# split training and test dataset
input_test = input[nb_train:,:]
targ_test = targ[nb_train:,:]

input = input[0:nb_train,:]
targ = targ[0:nb_train,:]


cnn.init(in_dim=i_ar([in_dim]), in_nb_ch=1, out_dim=3, \
		bias=-1.0, b_size=16, comp_meth="C_CUDA")

cnn.create_dataset("TRAIN", size=nb_train    , input=f_ar(input)     , target=f_ar(targ))
cnn.create_dataset("VALID", size=150-nb_train, input=f_ar(input_test), target=f_ar(targ_test))
cnn.create_dataset("TEST" , size=150-nb_train, input=f_ar(input_test), target=f_ar(targ_test))


cnn.dense(nb_neurons=8, activation="LOGI")
cnn.dense(nb_neurons=3, activation="SMAX")

cnn.train(nb_iter=1000, learning_rate=0.005, momentum=0.8, confmat=1, control_interv=100, save_every=100, silent=2)

cnn.perf_eval()

EOF


                   ..:^~!?JY5PB~                                                                                             
           J5PGB#&&&&&#BGP55YY#&#!                                                                                           
           &GGB##&&&@@@@@&#PJ?B#&B                                                                                           
          .&#&@@@@@@@@@@@@@@@B##&G                                                                                           
        ^G@@@@BJ^.   .~?P&@@@@@&&G                                                                                           
      &&@@@B^   :!???^^..:7&@@@@&G      .~~~         :~~:         ~~~^        ~~~    ~~~^        ~~~         ^~~.            
    .B@@@&^  !#@@@@@@@@&?^~J&@@@@B ^    !@@@.       :@@@@^       .@@@@G      .@@@:  .@@@@P      .@@@:       ~@@@@.           
   .&@@@#  .J@@@#GP55PB&@&JJ5@@@@B ^G   !@@@.       &@@@@@.      .@@@@@&:     @@@:  .@@@@@&:    .@@@.      :@@@@@&   